In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data=pd.read_csv('corona.csv')
data.sample(5)

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
5681,31/05/2020,31,5,2020,0,0,Brunei_Darussalam,BN,BRN,433296.0,Asia,0.000000
30807,25/04/2020,25,4,2020,22,0,Rwanda,RW,RWA,12626938.0,Africa,0.459335
17268,12/01/2020,12,1,2020,0,0,Iceland,IS,ISL,356991.0,Europe,NaN
13728,27/04/2020,27,4,2020,0,0,Gabon,GA,GAB,2172578.0,Africa,5.477364
26574,21/02/2020,21,2,2020,0,0,New_Zealand,NZ,NZL,4783062.0,Oceania,0.000000


In [3]:
data.columns

Index(['dateRep', 'day', 'month', 'year', 'cases', 'deaths',
       'countriesAndTerritories', 'geoId', 'countryterritoryCode',
       'popData2019', 'continentExp',
       'Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'],
      dtype='object')

In [4]:
data.drop(['dateRep','geoId','countryterritoryCode','continentExp','Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'],axis=1,inplace=True)

In [5]:
data['popData2019'].fillna(data['popData2019'].mean(),inplace=True)
data.isna().any()

day                        False
month                      False
year                       False
cases                      False
deaths                     False
countriesAndTerritories    False
popData2019                False
dtype: bool

In [7]:
data['popData2019']=data['popData2019']/1000000
data.sample(10)

,day,month,year,cases,deaths,countriesAndTerritories,popData2019
144,10,4,2020,61,1,Afghanistan,3.804176e-05
27595,23,8,2020,0,0,Norway,5.328212e-06
14990,21,1,2020,0,0,Greece,1.072460e-05
12364,6,6,2020,169,1,Ethiopia,1.120787e-04
26017,25,7,2020,191,0,Netherlands,1.728216e-05
28213,13,4,2020,336,7,Pakistan,2.165653e-04
29511,7,6,2020,567,16,Poland,3.797281e-05
2918,29,4,2020,88,0,Bahrain,1.641164e-06
38571,19,4,2020,2,1,United_States_Virgin_Islands,1.045790e-07
17862,27,5,2020,1787,57,Iran,8.291389e-05


In [8]:
data.rename({'popData2019':'Population', 
               'countriesAndTerritories':'Countries'},
              axis=1, inplace=True)
data.head()

,day,month,year,cases,deaths,Countries,Population
0,1,9,2020,34,4,Afghanistan,0.000038
1,31,8,2020,19,0,Afghanistan,0.000038
2,30,8,2020,3,0,Afghanistan,0.000038
3,29,8,2020,11,1,Afghanistan,0.000038
4,28,8,2020,3,0,Afghanistan,0.000038


In [9]:
index=data[data['cases']==0].index
data.drop(index,inplace=True)
index

Int64Index([    8,    13,    21,    22,    29,    30,    33,    50,   107,
              150,
            ...
            39939, 39940, 39942, 39943, 39945, 39946, 39949, 39951, 39952,
            39953],
           dtype='int64', length=14031)

In [9]:
data.describe()

,day,month,year,cases,deaths,Population
count,25925.000000,25925.000000,25925.000000,25925.000000,25925.000000,25925.000000
mean,16.235873,5.642392,2019.999961,983.958920,32.799383,53.378984
std,8.889216,1.735317,0.006211,4879.156962,154.133340,180.020102
min,1.000000,1.000000,2019.000000,-2461.000000,-1918.000000,0.000815
25%,9.000000,4.000000,2020.000000,9.000000,0.000000,4.043258
50%,16.000000,6.000000,2020.000000,58.000000,1.000000,10.724599
75%,24.000000,7.000000,2020.000000,360.000000,7.000000,37.411038
max,31.000000,12.000000,2020.000000,78761.000000,4928.000000,1433.783692


In [10]:
#data[data['deaths'].eq(float(-1918))]
data[data['deaths']<=-1]

,day,month,year,cases,deaths,Countries,Population
9705,6,7,2020,75,-3,Czechia,10.649800
9706,5,7,2020,121,-1,Czechia,10.649800
18975,25,6,2020,577,-31,Italy,60.359546
20677,24,8,2020,237,-443,Kyrgyzstan,6.415851
34115,12,8,2020,3172,-2,Spain,46.937060
34194,25,5,2020,-372,-1918,Spain,46.937060
34930,18,8,2020,63,-1,Sweden,10.230185


In [11]:
data[data['cases']==78761]

,day,month,year,cases,deaths,Countries,Population
17283,30,8,2020,78761,948,India,1366.417756


In [12]:
data['Countries'].value_counts()

China                          230
Singapore                      214
South_Korea                    209
Japan                          206
United_States_of_America       202
                              ... 
Holy_See                         9
Western_Sahara                   9
Montserrat                       7
Falkland_Islands_(Malvinas)      6
Anguilla                         2
Name: Countries, Length: 210, dtype: int64

 Finding the daily average of deaths & cases globally

In [13]:
from datetime import date
d1=date(2020,9,1)
d2=date(2019,3,25)
corona_days = (d1-d2).days

In [14]:
deaths_avg=data['deaths'].sum()/corona_days
cases_avg=data['cases'].sum()/corona_days
print(deaths_avg,'  ',cases_avg)

1616.5855513307986    48496.45437262357


In [15]:
data['cases'].sum()/1000000

25.509135

In [19]:
sum_by_month_country = data.groupby(['Countries','month'])[['deaths','cases']].agg([sum]).sort_index()
sum_by_month_country.loc[['Egypt','Netherlands']]

deaths  cases
                     sum    sum
Countries   month              
Egypt       2          0      1
            3         40    608
            4        340   4659
            5        533  18181
            6       1959  43305
            7       1902  27003
            8        625   4970
            9         22    212
Netherlands 2          0      2
            3        864  11748
            4       3847  27052
            5       1240   7455
            6        156   3966
            7         33   3740
            8         75  16108
            9          0    525

In [18]:
data[data['Countries']=='Egypt'].tail()

,day,month,year,cases,deaths,Countries,Population
11297,9,3,2020,34,1,Egypt,100.388076
11298,7,3,2020,12,0,Egypt,100.388076
11299,6,3,2020,1,0,Egypt,100.388076
11300,2,3,2020,1,0,Egypt,100.388076
11316,15,2,2020,1,0,Egypt,100.388076


In [45]:
corona_by_countries=data.groupby('Countries')
corona_by_countries[['cases','deaths']].si().head(10)

,cases,deaths
Countries,,
Afghanistan,38196,1405
Albania,9513,284
Algeria,44494,1510
Andorra,1176,48
Angola,2624,107
Anguilla,3,0
Antigua_and_Barbuda,94,2
Argentina,408413,8372
Armenia,43781,879


In [49]:
corona_by_countries=data.groupby('Countries')
corona_by_countries.sum().loc['Egypt']

day             2797.0000
month            978.0000
year          353500.0000
cases          98939.0000
deaths          5421.0000
Population     17567.9133
Name: Egypt, dtype: float64

In [43]:
corona_by_countries=data.groupby('Countries')
cases_by_countries=corona_by_countries['cases'].sum()
cases_by_countries.sort_values(ascending=False).head(50)

Countries
United_States_of_America    6031013
Brazil                      3908272
India                       3691166
Russia                       995319
Peru                         652037
South_Africa                 627041
Colombia                     615168
Mexico                       599560
Spain                        462858
Chile                        411726
Argentina                    408413
Iran                         375212
United_Kingdom               335873
Saudi_Arabia                 315772
Bangladesh                   312996
Pakistan                     296149
France                       281025
Turkey                       270133
Italy                        269214
Germany                      243599
Iraq                         234934
Philippines                  220819
Indonesia                    174796
Kazakhstan                   131596
Canada                       128948
Ukraine                      121215
Qatar                        118778
Israel            